In [77]:
import pandas as pd
import os

In [ ]:

base_path = "C:/LINCS_DATA/project/data/"
features_path = "C:/LINCS_DATA/project/features/"
zscore_file = base_path + "zscore_matrix_392.csv"

sig_df = pd.read_csv(base_path + "best_signature_per_drug_final.csv")
combo_file = base_path + "LABELS/combo_design.csv"
drug_file = base_path + "LABELS/drug.csv"
label_file = features_path + "pair_labels.csv"  



In [ ]:

zscore_df = pd.read_csv(zscore_file, index_col='sig_id')
combo_df = pd.read_csv(combo_file)
drug_df = pd.read_csv(drug_file)
sig_df = pd.read_csv(sig_file)
label_df = pd.read_csv(label_file)

In [ ]:

drug_map = dict(zip(drug_df['idDrug'], drug_df['name']))
combo_df['drugA_name'] = combo_df['idDrugA'].map(drug_map).str.lower().str.strip()
combo_df['drugB_name'] = combo_df['idDrugB'].map(drug_map).str.lower().str.strip()

In [ ]:

sig_df['pert_iname'] = sig_df['pert_iname'].str.lower().str.strip()
sig_map = dict(zip(sig_df['pert_iname'], sig_df['sig_id']))
combo_df['sig_id_A'] = combo_df['drugA_name'].map(sig_map)
combo_df['sig_id_B'] = combo_df['drugB_name'].map(sig_map)

In [ ]:

combo_df = combo_df.dropna(subset=['sig_id_A', 'sig_id_B'])

In [ ]:

combo_df = combo_df.reset_index(drop=True)
combo_df['row_index'] = combo_df.index + 1  # لمطابقة idSynergy_Score
label_map = dict(zip(label_df['idSynergy_Score'], label_df['zip']))
combo_df['zip'] = combo_df['row_index'].map(label_map)
combo_df = combo_df.dropna(subset=['zip'])


In [ ]:

combo_df['label'] = combo_df['zip'].apply(lambda x: 1 if x > -0.5 else 0)

In [ ]:

rows = []
n_genes = zscore_df.shape[1]
columns = (
    ['drugA_name', 'sig_id_A', 'drugB_name', 'sig_id_B'] +
    [f'gene{i+1}_A' for i in range(n_genes)] +
    [f'gene{i+1}_B' for i in range(n_genes)] +
    ['label']
)

for _, row in combo_df.iterrows():
    sigA, sigB = row['sig_id_A'], row['sig_id_B']
    if sigA in zscore_df.index and sigB in zscore_df.index:
        vecA = zscore_df.loc[sigA].values
        vecB = zscore_df.loc[sigB].values
        features = [
            row['drugA_name'], sigA,
            row['drugB_name'], sigB,
            *vecA, *vecB, int(row['label'])
        ]
        rows.append(features)

In [ ]:

df_final = pd.DataFrame(rows, columns=columns)
df_final.to_csv(os.path.join(features_path, "pair_feature_matrix_labeled_full.csv"), index=False)
print(f"✅ تم حفظ {len(df_final)} زوج دوائي في pair_feature_matrix_labeled_full.csv")


✅ تم حفظ 489 زوج دوائي في pair_feature_matrix_labeled_full.csv
